In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df = df.filter(['icustay_id','icu_day','chloride_max'])
df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
df_chl = df.query('icu_day == 2')
df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)
df_icd = df_icd.set_index('hadm_id')

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure',
       'liver_disease',  
        'coagulopathy',  'fluid_electrolyte',
       'blood_loss_anemia']
labs = ['chloride_max', 'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [4]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]
df_chl = df_chl['chl_110']

## standarize all values to z-score

In [5]:
stan = pd.read_csv('standard.csv')
stan = stan.set_index('vitalid')
stan

,mean,std
vitalid,,
DiasBP,59.431493,14.794056
HeartRate,82.917012,16.126807
RespRate,19.693027,5.462299
SysBP,123.279089,22.821488
BICARBONATE,25.696443,5.034271
CHLORIDE,104.144561,6.322994
SODIUM,138.852473,5.119334


In [6]:
df_vitals['heartrate_max'] = (df_vitals['heartrate_max'] - stan.loc['HeartRate']['mean'])/stan.loc['HeartRate']['std']
df_vitals['sysbp_min'] = (df_vitals['sysbp_min'] - stan.loc['SysBP']['mean'])/stan.loc['SysBP']['std']
df_vitals['diasbp_min'] = (df_vitals['diasbp_min'] - stan.loc['DiasBP']['mean'])/stan.loc['DiasBP']['std']
df_vitals['resprate_max'] = (df_vitals['resprate_max'] - stan.loc['RespRate']['mean'])/stan.loc['RespRate']['std']
df_vitals['weight'] = (df_vitals['weight'] - df_wt['weight'].mean())/df_wt['weight'].std()
df_vitals['mingcs'] = (df_vitals['mingcs'] - 13.66217630133403)/2.3355640788808962


# Labs

In [7]:
df_labs['chloride_max'] = (df_labs['chloride_max'] - stan.loc['CHLORIDE']['mean'])/stan.loc['CHLORIDE']['std']
df_labs['sodium_max'] = (df_labs['sodium_max'] - stan.loc['SODIUM']['mean'])/stan.loc['SODIUM']['std']
df_labs['bicarbonate_min'] = (df_labs['bicarbonate_min'] - stan.loc['BICARBONATE']['mean'])/stan.loc['BICARBONATE']['std']

df_labs.head()

,,,chloride_max,sodium_max,bicarbonate_min
subject_id,hadm_id,icustay_id,,,
1754,103405,279017,2.191278,0.810169,-1.528810
1798,179432,297430,2.507584,1.396183,-2.124725
3868,107365,261153,1.242361,0.419493,-2.522002
4708,152622,283689,0.767902,-0.361858,-1.330171
5384,186807,244571,2.349431,-0.361858,-1.528810


# Fluid

In [8]:
df_fluid['fluid_net_input_ml'] = (df_fluid['fluid_net_input_ml'] - df_fluid['fluid_net_input_ml'].mean())/ df_fluid['fluid_net_input_ml'].std()

# Age

In [10]:
df_demo['age'] = (df_demo['age'] - df_demo['age'].mean())/ df_demo['age'].std()

# Final

In [13]:
df = df_demo.join(df_chl, how = 'inner') \
.join(df_fluid, how = 'inner') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 

df = df.fillna(0)
df

age gender ethnicity  chl_110  \
subject_id hadm_id icustay_id                                       
4          185777  294638     -0.937959      F     WHITE        0   
6          107064  228232      0.110217      F     WHITE        0   
9          150750  220597     -1.288778      M     OTHER        0   
13         143045  263738     -1.400273      F     WHITE        1   
17         194023  277042     -0.960638      F     WHITE        0   
21         109451  217847      1.355531      M     WHITE        0   
           111970  216859      1.377893      M     WHITE        0   
26         197661  244882      0.461512      M     OTHER        0   
30         104557  225176      1.581533      M     OTHER        0   
31         128652  254478      0.476579      M     WHITE        0   
32         175413  295037     -1.073084      M     WHITE        0   
33         176176  296681      1.063234      M     OTHER        0   
36         165660  241249      0.503382      M     WHITE        0   
38         185910  248910      0.689577      M     WHITE        1   
41         101757  261027     -0.428858      M     WHITE        0   
42         119203  210828     -0.166378      M     OTHER        0   
43         146828  225852     -1.811359      M     WHITE        0   
44         181750  291554      0.935245      M     OTHER        0   
52         190797  261857     -1.501141      M     WHITE        0   
53         155385  239839     -1.070864      M     OTHER        0   
55         190665  262723      0.007128      F     WHITE        0   
56         181711  275642      1.581533      F     WHITE        0   
59         104130  224440      1.335547      F     WHITE        0   
61         176332  252348     -0.543525      M     WHITE        1   
           189535  217135     -0.511647      M     WHITE        1   
64         172056  232593     -2.163923      F     BLACK        0   
65         143430  244776     -2.170743      M     WHITE        0   
68         108329  272667     -1.285289      F     BLACK        0   
           170467  294232     -1.288461      F     BLACK        0   
71         111944  211832     -1.594873      F     ASIAN        0   
...                                 ...    ...       ...      ...   
99797      135425  225994     -0.363357      M     WHITE        0   
99810      196261  272409     -0.367639      M     WHITE        0   
99822      163117  220334     -0.161462      M     WHITE        0   
99823      179720  296047      1.035638      F     WHITE        0   
99836      101018  245746     -0.691338      F     OTHER        1   
99837      197195  253877     -2.238623      M     WHITE        0   
99847      155024  219261      0.671338      F     WHITE        0   
99862      178839  253685     -0.286278      F     OTHER        0   
99863      100749  216757     -1.236441      M     WHITE        0   
99868      177777  231346     -2.525527      F     OTHER        0   
99881      172327  214377      0.795203      M     WHITE        0   
99883      150755  276467      0.570311      M     WHITE        0   
99893      128349  208873      1.419921      M     WHITE        0   
99894      127380  204236     -0.233782      F     WHITE        1   
99897      162913  266801     -0.607440      M     BLACK        0   
99899      188409  245756      1.327617      M     BLACK        0   
99901      131711  226541      0.277538      M     WHITE        0   
99913      175989  292375      0.408540      M  HISPANIC        0   
99936      107913  213906      1.581533      F     WHITE        0   
99939      159023  252442     -2.148380      M  HISPANIC        0   
99944      185654  221067      1.242133      F     WHITE        0   
99955      108494  243255     -0.049015      M     BLACK        1   
99957      148574  257127      0.232496      M     WHITE        0   
99966      167228  252173      0.779026      M     WHITE        0   
99982      112748  275958      0.070885      M     WHITE        0   
           151454  221194      0.0

In [14]:
df.to_csv('dataFirst.csv')